In [37]:
import numpy as np
from scipy.stats import norm

def black_scholes(S, K, r, t, sigma, option_Type="Call", q=0):
    """
    Black-Scholes Option Pricing Model with Greeks.
    
    Parameters:
    S : float or array-like - Current stock price
    K : float or array-like - Strike price
    r : float - Risk-free interest rate (annualized)
    t : float - Time to expiration (in years)
    sigma : float - Volatility of the stock (annualized)
    option_Type : str - "Call" for call option, "Put" for put option
    q : float - Continuous dividend yield (default 0)

    Returns:
    tuple: (option price, delta, gamma, vega, theta, rho)
    """

    S, K, t, sigma = map(np.asarray, (S, K, t, sigma))

    if np.any(t <= 0):
        intrinsic_value = np.maximum(0, S - K) if option_Type == "Call" else np.maximum(0, K - S)
        return intrinsic_value, np.zeros_like(S), np.zeros_like(S), np.zeros_like(S), np.zeros_like(S), np.zeros_like(S)

    sqrt_t = np.sqrt(t)
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * t) / (sigma * sqrt_t)
    d2 = d1 - sigma * sqrt_t

    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    Nmd1 = norm.cdf(-d1)
    Nmd2 = norm.cdf(-d2)

    if option_Type == "Call":
        price = S * np.exp(-q * t) * Nd1 - K * np.exp(-r * t) * Nd2
        delta = np.exp(-q * t) * Nd1
        theta = (-S * sigma * np.exp(-q * t) * norm.pdf(d1)) / (2 * sqrt_t) - r * K * np.exp(-r * t) * Nd2 + q * S * np.exp(-q * t) * Nd1
        rho = K * t * np.exp(-r * t) * Nd2

    elif option_Type == "Put":
        price = K * np.exp(-r * t) * Nmd2 - S * np.exp(-q * t) * Nmd1
        delta = np.exp(-q * t) * Nmd1
        theta = (-S * sigma * np.exp(-q * t) * norm.pdf(d1)) / (2 * sqrt_t) + r * K * np.exp(-r * t) * Nmd2 - q * S * np.exp(-q * t) * Nmd1
        rho = -K * t * np.exp(-r * t) * Nmd2

    else:
        raise ValueError('Invalid option type. Choose "Call" or "Put".')

    gamma = (np.exp(-q * t) * norm.pdf(d1)) / (S * sigma * sqrt_t)
    vega = S * sqrt_t * np.exp(-q * t) * norm.pdf(d1)

    return price, delta, gamma, vega, theta, rho


In [38]:
def monte_carlo_european(S0, K, T, r, sigma, option_type="Call", num_simulations=10000, num_steps=252):
    '''Monte Carlo Simulation to price a European Call or Put option using GBM
    Params:
    S0: inital stock price
    K:Strike Price
    T:Time to expiration(years)
    r:risk free interest rate
    sigma: Volatility of stock
    option_type: "Call" for call option, "Put" for put option"
    num_simulations: number of Monte Carlo Simulations
    num_steps: Number of time steps in simulation
    Returns:
    float: Estimated option price
    '''
    dt=T/num_steps
    np.random.seed(42)

    Z=np.random.standard_normal((num_simulations,num_steps))
    S=np.zeros((num_simulations,num_steps))
    S[:,0]=S0
    for t in range(1,num_steps):
        S[:, t] = S[:, t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t])

    #Compute option payout when it expires cuz why not

    ST=S[:,-1]
    if option_type=="Call":
        payoff=np.maximum(ST-K,0)
    elif option_type=="Put":
        payoff=np.maximum(K-ST,0)
    else:
        raise ValueError("Invalid Option Type: Choose 'Put' or 'Call'")
    
    option_price=np.exp(-r*T)*np.mean(payoff)
    return option_price
    
    
    
    

In [39]:
S = np.array([100, 105, 110]) 
K = 110   
T = 1     
r = 0.05  
sigma = 0.2  
q = 0.02  

call_price, call_delta, call_gamma, call_vega, call_theta, call_rho = black_scholes(S, K, T, r, sigma, option_Type="Call", q=q)
put_price, put_delta, put_gamma, put_vega, put_theta, put_rho = black_scholes(S, K, T, r, sigma, option_Type="Put", q=q)

print(f"Call Option Prices: {call_price}")
print(f"Put Option Prices: {put_price}")
print(f"Call Delta: {call_delta}, Put Delta: {put_delta}")
print(f"Gamma: {call_gamma}")
print(f"Vega: {call_vega}")
print(f"Call Theta: {call_theta}, Put Theta: {put_theta}")
print(f"Call Rho: {call_rho}, Put Rho: {put_rho}")


Call Option Prices: [0.35580583 2.00176485 5.58665301]
Put Option Prices: [5.09099255 1.74194906 0.33183472]
Call Delta: [0.15533493 0.53048157 0.86735597], Put Delta: [0.84366557 0.46851893 0.13164453]
Gamma: [0.05334059 0.0846168  0.04336447]
Vega: [5.33405863 9.32900261 5.2471008 ]
Call Theta: [-25.53513414 -71.2427942  -98.40852233], Put Theta: [77.10210156 31.29454145  4.02891327]
Call Rho: [0.75888434 2.68494001 4.49112519], Put Rho: [-4.4728775  -2.54682182 -0.74063664]


In [40]:
S0 = 100    
K = 110     
T = 1      
r = 0.05   
sigma = 0.2 

call_price = monte_carlo_european(S0, K, T, r, sigma, option_type="Call")
put_price = monte_carlo_european(S0, K, T, r, sigma, option_type="Put")

print(f"Monte Carlo Call Option Price: {call_price:.4f}")
print(f"Monte Carlo Put Option Price: {put_price:.4f}")


Monte Carlo Call Option Price: 5.7990
Monte Carlo Put Option Price: 10.6474
